###  кратко
Предикты получал произведением выхода классификатора и сегментатора( так получался наибольший скор), для постобработки использовал общедоступные функции удаления небольших областей + convex hull для большего охвата масок. Почему-то блендинг у меня не зашел, а про TTA я вспомнил за несколько часов до конца соревнования и не успел его нормально написать. 

### UNet Inference kernel


In [3]:
from segmentation_models_pytorch import Unet

import pdb
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset
from albumentations import (Normalize, Compose, Resize)
from albumentations.pytorch import ToTensor
import torch.utils.data as data
import torch.nn.functional as F
# from model import Unet

In [4]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, mean, std):
        self.root = root
        df['Image'] = df['Image_Label'].apply(lambda x: x.split('_')[0])
        self.fnames = df['Image'].unique().tolist()
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Resize(512,768),
                Normalize(mean=mean, std=std, p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return fname, images

    def __len__(self):
        return self.num_samples

In [7]:
#deleting small masks
def post_process(probability, threshold, min_size):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    mask = cv2.resize(mask, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [8]:
# modifying masks for bigger square via convex hull
def draw_convex_hull(mask, mode='convex'):
    mask = np.array(mask, dtype = np.uint8)
    img = np.zeros(mask.shape)
    contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if mode=='rect': # simple rectangle
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), -1)
        elif mode=='convex': # minimum convex hull
            hull = cv2.convexHull(c)
            cv2.drawContours(img, [hull], 0, (255, 255, 255),-1)
        elif mode=='approx':
            epsilon = 0.02*cv2.arcLength(c,True)
            approx = cv2.approxPolyDP(c,epsilon,True)
            cv2.drawContours(img, [approx], 0, (255, 255, 255),-1)
        else: # minimum area rectangle
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (255, 255, 255),-1)
    return img/255.

In [9]:
sample_submission_path = './data/sample_submission.csv'
test_data_folder = "./data/test_images"

In [11]:
# initialize test dataloader
best_threshold = [0.3,0.4,0.3,0.3]#resnet
num_workers = 2
batch_size = 4
print('best_threshold', best_threshold)
min_size = 11000
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
df = pd.read_csv(sample_submission_path)
testset = DataLoader(
    TestDataset(test_data_folder, df, mean, std),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

best_threshold [0.3, 0.3, 0.3, 0.3]


In [18]:
ckpt_path2 = "./model/Unet34_bcedice_enc_dec_all_pic.pth"
device = torch.device("cuda:0")
model = Unet("resnet34", encoder_weights=None, classes=4,activation=None,attention_type='scse',center=True)
model.to(device)
model.eval()
state = torch.load(ckpt_path2, map_location=lambda storage, loc: storage)
model.load_state_dict(state['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [19]:
from torchvision.models import resnet34

In [20]:
from efficientnet_pytorch import EfficientNet

In [22]:
class_ckpt_path = "./model/classification_model_resnet34_all_data.pth"
device = torch.device("cuda:0")
cl_model = resnet34()
in_feats = cl_model.fc.in_features
cl_model.fc = torch.nn.Linear(in_features=in_feats,out_features=4,bias = True)
cl_model.to(device)
cl_model.eval()
cl_state = torch.load(class_ckpt_path) 
cl_model.load_state_dict(cl_state['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
# start prediction
predictions = []
mapping = {0:'Fish',1:'Flower',2:'Gravel',3:'Sugar'}
for i, batch in enumerate(tqdm(testset)):
    fnames, images = batch
    cl_preds = torch.sigmoid(cl_model(images.to(device)))
    batch_preds = torch.sigmoid(model(images.to(device)))
    batch_preds = cl_preds[:,:,None,None]*batch_preds
    batch_preds = batch_preds.detach().cpu().numpy()
    for fname, preds in zip(fnames, batch_preds):
        for cls, pred in enumerate(preds):
            pred, num = post_process(pred, best_threshold[cls], min_size)
            pred = draw_convex_hull(pred)
            rle = mask2rle(pred)
            name = fname + f"_{mapping[cls]}"
            predictions.append([name, rle])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['Image_Label', 'EncodedPixels'])
df.to_csv("./submissions/Unet34_bce_dice_submission_11k_connected_resnet_class.csv", index=False)

100%|██████████| 925/925 [03:03<00:00,  5.03it/s]
